In [1]:
%pylab inline

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
import jax
import jax.numpy as jnp
from jax import random

import sys
sys.path.append('wl-mechanics')
import datasets
from models import utils as mutils
from models import mlp
import train_utils as tutils
from flax.training import checkpoints

2023-11-16 22:57:46.633812: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /pkgs/cuda-11.8/targets/x86_64-linux/lib:/pkgs/cudnn-8.8/lib:/pkgs/cuda-11.3/lib64:/pkgs/nccl_2.9.9-1+cuda11.3_x86_64:/pkgs/nccl_2.8.3-1+cuda11.0_x86_64/lib:/pkgs/cuda-11.8/targets/x86_64-linux/lib:/pkgs/cudnn-8.8/lib
2023-11-16 22:57:46.634637: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /pkgs/cuda-11.8/targets/x86_64-linux/lib:/pkgs/cudnn-8.8/lib:/pkgs/cuda-11.3/lib64:/pkgs/nccl_2.9.9-1+cuda11.3_x86_64:/pkgs/nccl_2.8.3-1+cuda11.0_x86_64/lib:/pkgs/cuda-11.8/targets/x86_64-linux/lib:/pkgs/cudnn-8.8/lib
2023-11-16 22:57:46.634662: W tensor

In [3]:
!ls wl-mechanics/assets

ebdata_v3.h5ad		       test_4i.npz
op_cite_inputs_0.h5ad	       test_embrio.npz
op_train_multi_targets_0.h5ad  train_4i.npz
ot.pdf			       train_embrio.npz
ot.png			       ubot.pdf
phot.pdf		       ubot.png
phot.png		       v2_full-4i_normalized.csv
sb.pdf			       v2_full-4i_train_test_split_idxs.npz
sb.png			       val_embrio.npz


In [4]:
with np.load('wl-mechanics/assets/v2_full-4i_train_test_split_idxs.npz') as data:
  print(data.files)
  print(data['train_idxs'], len(data['train_idxs']))
  print(data['test_idxs'], len(data['test_idxs']))
  train_ids = data['train_idxs']
  test_ids = data['test_idxs']

['train_idxs', 'test_idxs']
[165900 122300  32952 ...   9404 177057  63966] 254148
[ 64517 177348  56344 ... 194509 316355 238922] 63537


In [5]:
import pandas as pd

df = pd.read_csv('wl-mechanics/assets/v2_full-4i_normalized.csv')
df.head()

,time,Nuclei_Morphology_Area,Nuclei_Morphology_Circularity,Nuclei_Morphology_Convexity,Nuclei_Morphology_Eccentricity,Nuclei_Morphology_Elongation,Nuclei_Morphology_Equivalent_Diameter,Nuclei_Morphology_Extent,Nuclei_Morphology_Major_Axis_Length,Nuclei_Morphology_Mean_Radius,...,Intensity_mean_pAKT,Intensity_mean_pEGFR,Intensity_mean_aTUB,Intensity_mean_pERK,Intensity_mean_Ki67,Intensity_mean_pMET,Intensity_mean_CD45,Intensity_mean_ClCasp3,Intensity_mean_TotProtein,Condition
0,8,-0.274112,-1.161924,-1.024294,-2.117290,-1.385679,-0.282769,-0.084229,-0.867892,-0.119449,...,2.740725,2.050947,1.256621,1.159009,1.124681,3.885301,1.724430,4.769124,0.986931,ixazomib
1,8,-1.140102,-0.974085,-2.296770,-1.142132,-0.906933,-1.328908,-1.082079,-1.446597,-1.284489,...,0.591499,-0.120799,-0.599442,0.443334,-0.119510,1.289209,0.819025,2.304385,0.167991,ixazomib
2,8,0.232487,-2.525538,-3.063312,-2.752453,-1.588989,0.227103,-0.612237,-0.467192,-0.390752,...,1.763594,2.121434,0.368434,0.297882,1.091764,3.158411,3.025618,4.459429,1.128970,ixazomib
3,8,1.428426,-2.658096,-2.244442,-0.976498,-0.802136,1.259917,-1.410106,0.608182,0.698723,...,0.704826,1.046160,-0.100396,0.284098,0.472126,1.877242,1.778915,2.808882,0.744667,ixazomib
4,8,2.323858,-2.901381,-3.469861,-1.247588,-0.969757,1.928539,-1.712069,1.127107,1.269271,...,0.843195,0.548065,-0.587789,0.705410,0.499171,2.220354,2.300006,3.848469,0.693036,ixazomib


In [6]:
np.unique(df['time'])

array([ 8, 24, 48])

In [7]:
drugs = 'trametinib_erlotinib'
# df = df[df['Condition'] == drugs]
# df

In [19]:
train_df = df.iloc[train_ids]
train_df = train_df[train_df['Condition'] == drugs]
val_df = train_df.iloc[np.random.choice(len(train_df), 1000, replace=False)]
test_df = df.iloc[test_ids]
test_df = test_df[test_df['Condition'] == drugs]
features = list(filter(lambda s: s != 'time' and s != 'Condition', df.columns))

In [20]:
def save_numpy(df, name):
  X = df[features].to_numpy()
  t = df['time'].to_numpy()
  np.savez(f'wl-mechanics/assets/{name}_4i.npz', X=X, ts=t)

In [21]:
save_numpy(train_df, 'train')
save_numpy(val_df, 'val')
save_numpy(test_df, 'test')

In [22]:
val_df

,time,Nuclei_Morphology_Area,Nuclei_Morphology_Circularity,Nuclei_Morphology_Convexity,Nuclei_Morphology_Eccentricity,Nuclei_Morphology_Elongation,Nuclei_Morphology_Equivalent_Diameter,Nuclei_Morphology_Extent,Nuclei_Morphology_Major_Axis_Length,Nuclei_Morphology_Mean_Radius,...,Intensity_mean_pAKT,Intensity_mean_pEGFR,Intensity_mean_aTUB,Intensity_mean_pERK,Intensity_mean_Ki67,Intensity_mean_pMET,Intensity_mean_CD45,Intensity_mean_ClCasp3,Intensity_mean_TotProtein,Condition
178715,24,0.106599,-1.084040,-0.528961,-0.256009,-0.245753,0.105466,-0.770606,-0.102526,0.088115,...,3.288893,0.013928,0.318092,-0.188659,0.045889,-0.017199,-0.009561,-0.218826,-0.173393,trametinib_erlotinib
294139,48,2.407107,-1.198517,-1.735799,0.572473,0.680211,1.987301,-1.127095,2.419034,0.890756,...,-0.270605,-0.659487,-0.416262,-0.204056,-0.587652,-0.939749,-0.467970,-0.661947,-0.646881,trametinib_erlotinib
112143,24,-1.228426,0.755796,0.486999,0.168205,0.178844,-1.454557,0.674454,-1.313883,-1.074927,...,-0.310512,-0.500434,1.248626,-0.211290,0.182418,-0.393597,-0.497840,-0.553563,0.184783,trametinib_erlotinib
112089,24,-0.201015,-0.659578,-1.369740,0.058553,0.060555,-0.205621,-0.618095,-0.204020,-0.243422,...,-0.417168,-0.418095,-0.608661,-0.211274,-0.482743,-0.675267,-0.382926,-0.523685,-0.603285,trametinib_erlotinib
179118,24,0.769543,0.566387,0.384051,-0.560598,-0.502799,0.715370,0.342940,0.288114,0.931191,...,1.927283,0.275076,-0.163957,-0.207075,0.252699,-0.077697,-0.180494,-0.323031,-0.190461,trametinib_erlotinib
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294092,48,1.043655,-0.489972,-0.620366,0.204768,0.219781,0.948306,-0.689368,0.911565,0.748560,...,-0.444985,-0.573890,-0.101664,-0.203490,-0.494840,-0.752913,-0.384293,-0.616366,-0.532592,trametinib_erlotinib
161614,24,0.582741,0.913484,0.818840,0.054915,0.056744,0.550699,0.951304,0.425165,0.805580,...,-0.340261,0.155849,0.889945,-0.217325,0.286034,0.017111,-0.055508,-0.180895,0.444789,trametinib_erlotinib
76580,8,0.087310,-0.356306,0.085397,-0.049801,-0.050187,0.086554,0.606696,-0.036671,0.244805,...,-0.152657,-0.223259,0.047698,-0.173601,-0.259046,-0.516987,-0.193116,-0.403409,-0.457272,trametinib_erlotinib
178780,24,-0.084264,-0.213894,-1.080568,0.483532,0.559793,-0.085077,-0.261275,0.275390,-0.384250,...,2.670602,-0.228067,0.101521,-0.207039,-0.149082,-0.285899,-0.373466,-0.479985,-0.503311,trametinib_erlotinib


In [9]:
test_df

,time,Nuclei_Morphology_Area,Nuclei_Morphology_Circularity,Nuclei_Morphology_Convexity,Nuclei_Morphology_Eccentricity,Nuclei_Morphology_Elongation,Nuclei_Morphology_Equivalent_Diameter,Nuclei_Morphology_Extent,Nuclei_Morphology_Major_Axis_Length,Nuclei_Morphology_Mean_Radius,...,Intensity_mean_pAKT,Intensity_mean_pEGFR,Intensity_mean_aTUB,Intensity_mean_pERK,Intensity_mean_Ki67,Intensity_mean_pMET,Intensity_mean_CD45,Intensity_mean_ClCasp3,Intensity_mean_TotProtein,Condition
275798,48,0.216244,-0.701630,-0.405460,0.116111,0.121832,0.211578,-0.145978,0.172908,0.254796,...,-0.467107,-0.493395,-0.297149,-0.215123,-0.440421,-0.617635,-0.527239,-0.649984,-0.455227,trametinib_erlotinib
76402,8,-0.659898,-0.400738,-0.096410,0.851261,1.106170,-0.714665,-0.114373,-0.110372,-0.797019,...,0.186040,-0.091303,0.014798,-0.169418,-0.262193,0.149671,0.020990,0.252864,-0.274722,trametinib_erlotinib
293827,48,-0.705584,-0.964579,-0.284755,0.914122,1.214748,-0.768959,-1.525292,-0.083902,-0.958794,...,0.036849,0.237000,1.580126,-0.204042,0.253853,-0.124293,0.216668,-0.190139,0.432071,trametinib_erlotinib
161648,24,-0.146193,-1.630370,-4.041432,0.064740,0.067054,-0.148619,-1.819371,0.051651,-1.126005,...,-0.135134,-0.727941,-0.232038,-0.217327,-0.482402,-0.663027,-0.562821,-0.698900,-0.370735,trametinib_erlotinib
76034,8,1.630457,-2.880856,-5.862187,0.610025,0.733058,1.417172,-3.393056,2.451823,0.496502,...,0.093129,1.408854,0.222077,-0.098458,1.092290,0.787201,2.582427,1.176022,1.621623,trametinib_erlotinib
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162359,24,-0.376650,0.919171,0.350688,-0.174516,-0.170722,-0.393325,0.512350,-0.517515,-0.071613,...,-0.420381,-0.660219,-0.145264,-0.217305,-0.249169,-0.510716,-0.518984,-0.641574,-0.135489,trametinib_erlotinib
112665,24,-0.049746,0.319783,-0.055655,-0.407746,-0.378215,-0.050037,0.368632,-0.301800,0.215695,...,-0.299189,0.532660,1.038945,-0.211279,0.394505,0.132946,0.341788,-0.041571,0.214764,trametinib_erlotinib
218702,48,0.573604,-0.277104,0.187579,0.252714,0.274651,0.542512,-0.344015,0.555084,0.566953,...,-0.461702,-0.729653,0.546645,-0.211210,-0.540501,-0.525967,-0.599270,-0.642962,-0.371642,trametinib_erlotinib
178750,24,-0.676142,0.511200,0.575459,0.251695,0.273470,-0.733885,-0.044382,-0.611794,-0.501223,...,2.597780,-0.302810,0.683340,-0.202567,-0.065888,0.251196,-0.325902,-0.307387,-0.514849,trametinib_erlotinib


In [10]:
train_df

,time,Nuclei_Morphology_Area,Nuclei_Morphology_Circularity,Nuclei_Morphology_Convexity,Nuclei_Morphology_Eccentricity,Nuclei_Morphology_Elongation,Nuclei_Morphology_Equivalent_Diameter,Nuclei_Morphology_Extent,Nuclei_Morphology_Major_Axis_Length,Nuclei_Morphology_Mean_Radius,...,Intensity_mean_pAKT,Intensity_mean_pEGFR,Intensity_mean_aTUB,Intensity_mean_pERK,Intensity_mean_Ki67,Intensity_mean_pMET,Intensity_mean_CD45,Intensity_mean_ClCasp3,Intensity_mean_TotProtein,Condition
275633,48,0.062944,-0.658394,-0.369858,-0.787760,-0.673049,0.062558,0.097627,-0.347400,0.279222,...,-0.435127,-0.538165,-0.524588,-0.215130,-0.346497,-0.784790,-0.461487,-0.673533,-0.288049,trametinib_erlotinib
112498,24,-1.303553,-0.741629,-0.536323,1.198309,1.790715,-1.565404,1.054265,-0.517865,-1.761100,...,-0.443802,-0.301336,0.523970,-0.211285,-0.381492,0.012531,-0.441819,-0.167296,0.240434,trametinib_erlotinib
112839,24,-1.763452,0.546494,-0.288348,-0.615744,-0.545707,-2.356001,-0.922979,-2.279599,-1.988576,...,-0.387436,-0.437237,0.001319,-0.211234,0.096048,-0.477706,-0.223582,-0.472777,-0.335657,trametinib_erlotinib
161371,24,0.264975,0.474076,0.560441,-0.609166,-0.540640,0.258009,0.445212,-0.124112,0.559540,...,-0.321071,0.285090,0.759071,-0.217101,0.360318,0.352539,0.096049,0.076231,0.364122,trametinib_erlotinib
275788,48,-1.563452,-0.418477,-0.485998,-0.000907,-0.000937,-1.983663,0.343712,-1.784992,-1.735494,...,0.094690,0.321196,-0.551626,-0.215132,-0.038196,-0.504033,-0.151441,-0.408550,0.730899,trametinib_erlotinib
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112556,24,-0.101523,-0.595075,-1.902382,0.108508,0.113640,-0.102697,-1.793155,0.035421,-0.623868,...,-0.333507,-0.000038,-0.201443,-0.211270,0.457797,0.135082,-0.278018,-0.227338,1.080551,trametinib_erlotinib
293964,48,0.535025,-0.524075,-0.193027,0.929755,1.242617,0.507801,-0.487692,1.312271,0.125912,...,-0.430162,0.159233,-0.158983,-0.204060,-0.524252,-0.473789,-0.494141,-0.550030,-0.262814,trametinib_erlotinib
61693,8,0.269036,-0.247288,-0.044789,0.606381,0.727874,0.261858,-1.307007,0.629483,0.122064,...,-0.337067,-0.181845,-0.388283,-0.195236,-0.112785,-0.546234,0.069175,-0.437667,-0.165593,trametinib_erlotinib
61430,8,0.035533,-0.279688,-0.169410,0.113996,0.119551,0.035417,0.118866,0.013880,0.155712,...,-0.086957,0.094996,0.883797,-0.166792,0.202249,-0.511002,0.151831,-0.330463,-0.001980,trametinib_erlotinib


In [11]:
X[:2], t[:2]

NameError: name 'X' is not defined

In [11]:
with np.load('wl-mechanics/assets/test_rna.npz') as data:
  print(data.files)

['X', 'ts', 'v', 'idx']


In [15]:
with np.load('wl-mechanics/assets/train_rna.npz') as data:
  X = [data['X'][data['ts'] == t] for t in np.unique(data['ts'])]
for i in range(len(X)):
  print(X[i].shape)
  
for i in range(len(X)):
  print(350/(X[i].shape[0] + 350))

(1831, 100)
(3613, 100)
(2728, 100)
(3115, 100)
(2782, 100)
0.16047684548372307
0.08831693161746151
0.11371020142949967
0.10101010101010101
0.11174968071519796
